In [1]:
from tqdm import tqdm
import gym

from yaw_planner import Oxford


env = gym.make('gym-2d-perception-v0')
# policy = LookAhead(env.dt)
policy = Oxford(env.dt, env.dim)
# plt.ion()
max_step = 10000
rewards = []
steps = []
success = 0
fail = 0
for i in tqdm(range(max_step)):
    
    a = policy.plan(env.observation)
    observation, reward, done= env.step(a)
    if reward == 100:
        success += 1
    if reward == -100:
        fail += 1
    if done:
        env.reset()

    rewards.append(reward)
    steps.append(i)

    env.render()

pygame 2.1.2 (SDL 2.0.16, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


  0%|          | 0/10000 [00:00<?, ?it/s]/home/smoggy/thesis/gym-Drone2D-ActivePerception/yaw_planner.py:69: RuntimeWarning: invalid value encountered in divide
  view_map = np.where(np.arccos(((x - drone.x)*vec_yaw[0] + (y - drone.y)*vec_yaw[1]) / np.sqrt((drone.x - x)**2 + (drone.y - y)**2)) <= view_angle, np.where(((drone.x - x)**2 + (drone.y - y)**2 <= drone.yaw_depth ** 2), 1, 0), 0)
 86%|████████▌ | 8553/10000 [04:49<00:48, 29.56it/s]


TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'

In [2]:
print(success / (success + fail))

0.9069767441860465


In [4]:
from yaw_planner import LookAhead

env = gym.make('gym-2d-perception-v0')
env.render = False
policy = LookAhead(env.dt)
# policy = Oxford(env.dt, env.dim)
# plt.ion()
max_step = 10000
rewards = []
steps = []
success = 0
fail = 0
for i in tqdm(range(max_step)):
    
    a = policy.plan(env.observation)
    observation, reward, done= env.step(a)
    if reward == 100:
        success += 1
    if reward == -100:
        fail += 1
    if done:
        env.reset()

    rewards.append(reward)
    steps.append(i)

100%|██████████| 10000/10000 [03:54<00:00, 42.68it/s]


In [5]:
print(success / (success + fail))

0.9310344827586207


In [6]:
from yaw_planner import NoControl

env = gym.make('gym-2d-perception-v0')
env.render = False
policy = NoControl()
# policy = Oxford(env.dt, env.dim)
# plt.ion()
max_step = 10000
rewards = []
steps = []
success = 0
fail = 0
for i in tqdm(range(max_step)):
    
    a = policy.plan(env.observation)
    observation, reward, done= env.step(a)
    if reward == 100:
        success += 1
    if reward == -100:
        fail += 1
    if done:
        env.reset()

    rewards.append(reward)
    steps.append(i)

TypeError: __init__() takes 1 positional argument but 2 were given

In [7]:
print(success / (success + fail))

0.9310344827586207
